In [ ]:
%load_ext autoreload
%autoreload 2
%run -n main.py

# sources

In [ ]:
# path = join_path(DATA_DIR, SOURCE)
# !mkdir -p {path}

In [ ]:
# for name in [NE5, BSNLP, FACTRU, GAREEV]:
#     path = join_path(CORUS_DATA_DIR, CORUS_FILES[name])
#     records = LOADS[name](path)
#     records = (_.adapted for _ in records)
#     records = log_progress(records, desc=name)

#     path = join_path(DATA_DIR, TARGET, name + JL + GZ)
#     items = (_.as_json for _ in records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
%run -n main.py
sources = {}
for name in [NE5, BSNLP, FACTRU, GAREEV]:
    path = join_path(DATA_DIR, SOURCE, name + JL + GZ)
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    sources[name] = [Markup.from_json(_) for _ in items]

# annotators

## cpu

In [ ]:
# for name in [MITIE, NATASHA, PULLENTI, TEXTERRA, TOMITA, DEEPPAVLOV, DEEPPAVLOV_BERT]:
#     path = join_path(DATA_DIR, name)
#     !mkdir -p {path}

In [ ]:
# %run -n main.py
# docker = docker_client()

In [ ]:
# %run -n main.py
# for ann_name in [MITIE, TOMITA, NATASHA, PULLENTI, TEXTERRA]:
#     log(ann_name)
#     ann = ANNS[ann_name]()
#     ann.start(docker)
#     ann.wait()

#     for source_name in [NE5, BSNLP, FACTRU, GAREEV]:
#         records = ann.map(_.text for _ in sources[source_name])
#         records = (_.adapted for _ in records)
#         records = log_progress(records, desc=source_name)

#         path = join_path(DATA_DIR, ann_name, source_name + JL + GZ)
#         items = (_.as_json for _ in records)
#         lines = format_jl(items)
#         dump_gz_lines(lines, path)
    
#     ann.stop(docker)

## gpu

In [ ]:
# for deeppavlov anns rent gpu
# select cuda 10.0, high bandwith
# !vast search offers | grep '1 x  GTX 1080 Ti'

In [ ]:
# %run -n main
# # ann = DeeppavlovAnnotator()
# ann = DeeppavlovBERTAnnotator()

In [ ]:
# !vast create instance 488497 --image {ann.image} --disk 20

In [ ]:
# !vast show instances

In [ ]:
# !ssh ssh4.vast.ai -p 12870 -l root -Nf -L {ann.port}:localhost:{ann.container_port}

In [ ]:
# for source_name in [NE5, BSNLP, FACTRU, GAREEV]:
#     records = sources[source_name]
#     records = log_progress(records, desc=source_name)
#     records = ann.map(_.text for _ in records)
#     records = (_.adapted for _ in records)

#     path = join_path(DATA_DIR, ann.name, source_name + JL + GZ)
#     items = (_.as_json for _ in records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
# !vast destroy instance 492870

# score

In [ ]:
source_anns = {}
for source in SOURCES:
    for ann in ANNS:
        path = join_path(DATA_DIR, ann, source + JL + GZ)
        lines = load_gz_lines(path)
        items = parse_jl(lines)
        source_anns[source, ann] = [Markup.from_json(_) for _ in items]

In [ ]:
scores = {}
for source, ann in log_progress(source_anns):
    preds = source_anns[source, ann]
    targets = sources[source]
    scores[source, ann] = score_markups(preds, targets)

In [ ]:
table = scores_report_table(scores, SOURCES, ANNS)
output = format_scores_report(table)
output

In [ ]:
output = format_github_scores_report(table)
html = output.to_html(escape=False)
html = html.replace('border="1"', 'border="0"')
HTML(html)

# bench

In [ ]:
BENCH = [
    # GTX 1080 Ti
    Bench(
        DEEPPAVLOV,
        init=5.9,
        disk=1 * GB,  # 1GB emb + 5MB model
        ram=3 * GB,
        speed=24.31,  # 1.95 / 7 cores on CPU,
        device=GPU
    ),
    Bench(
        DEEPPAVLOV_BERT,
        init=34.5,
        disk=2 * GB,
        ram=6 * GB,
        speed=13.13,  # 17.71 / 3 cores on CPU
        device=GPU
    ),
    
    # 16 CPUs
    Bench(
        PULLENTI,
        init=2.85,
        disk=16 * MB,
        ram=253 * MB,
        speed=6.05
    ),
    Bench(
        TEXTERRA,
        init=47.6,
        disk=193 * MB,
        ram=3.3 * GB,  # leaks
        speed=20.16 / 5  # utils ~5 cores
    ),
    Bench(
        TOMITA,
        init=2.03,
        disk=64 * MB,
        ram=63 * MB,
        speed=29.8,
    ),
    Bench(
        NATASHA,
        init=2.05,
        disk=700 * KB,
        ram=160 * MB,
        speed=8.8,
    ),
    Bench(
        MITIE,
        init=28.3,
        disk=327 * MB,
        ram=261 * MB,
        speed=32.8,
    ),    
]

output = format_bench_report(BENCH, ANNS)
html = output.to_html(escape=False)
html = html.replace('border="1"', 'border="0"')
HTML(html)